In [19]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import pytorch
import torch
import torchrec
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [20]:
#load data
train = pd.read_csv('ml-100k/u.data', sep='\t', header=None)

In [21]:
#plot data
train.head()

,0,1,2,3
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [22]:
#make a recommendation system
#make a class for the dataset
class MovieDataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return torch.tensor(self.data.iloc[idx, :3].values, dtype=torch.long)

#make a class for the model
class Recommender(nn.Module):
    def __init__(self, n_users, n_movies, n_factors=50):
        super().__init__()
        self.user_factors = nn.Embedding(n_users, n_factors)
        self.movie_factors = nn.Embedding(n_movies, n_factors)
    def forward(self, user, movie):
        return (self.user_factors(user) * self.movie_factors(movie)).sum(1)

#make a function to train the model
def train_model(model, train_dl, lr=0.01, epochs=10):
    opt = optim.SGD(model.parameters(), lr=lr)
    loss_func = nn.MSELoss()
    for epoch in range(epochs):
        for users, movies, ratings, timestamp in train_dl:
            preds = model(users, movies)
            loss = loss_func(preds, ratings.float())
            loss.backward()
            opt.step()
            opt.zero_grad()
        print(loss.item())

#make a function to predict the ratings
def predict(model, test_dl):
    preds = []
    for users, movies, ratings, timestamp in test_dl:
        preds.append(model(users, movies))
    return torch.cat(preds).detach().numpy()

#make a function to evaluate the model
def evaluate(model, test_dl):
    preds = predict(model, test_dl)
    return np.sqrt(np.mean((preds - test_dl.dataset.data[:, 2])**2))

In [23]:
#train the model
train_ds = MovieDataset(train)
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
model = Recommender(943, 1682)
train_model(model, train_dl)


ValueError: too many values to unpack (expected 3)